In [ ]:
import sqlite3
import pandas as pd
import datetime
from datetime import datetime

In [ ]:
connection = sqlite3.connect('flights.db')
cursor = connection.cursor()

In [ ]:
cursor.execute('''CREATE TABLE FlightLeg (
    id INT PRIMARY KEY NOT NULL, 
    tailNumber text NOT NULL,
    sourceAirportCode CHAR(3),
    sourceCountryCode CHAR(3),
    destinationAirportCode CHAR(3),
    destinationCountryCode CHAR(3),
    departureTimeUtc TEXT,
    landingTimeUtc TEXT);''')

In [ ]:
url = "https://bitpeak.pl/datasets/flightlegs.csv"
flightlegs = pd.read_csv(url)
flightlegs

In [ ]:
insert_data = "INSERT INTO FlightLeg (id,tailNumber,sourceAirportCode,sourceCountryCode,destinationAirportCode,destinationCountryCode,departureTimeUtc,landingTimeUtc) \
      VALUES (?,?,?,?,?,?,?,? )"
for i in range(len(flightlegs.index)):
    string = flightlegs.iloc[i][0]
    data = string.split(";")
    data_tuple = (i,data[0],data[1],data[2],data[3],data[4],data[5],data[6])
    cursor.execute(insert_data, data_tuple)
    connection.commit()

In [ ]:
#cursor.execute('SELECT * FROM FlightLeg')
#for row in cursor.fetchall():
#     print(row)

In [ ]:
addColumn_flightDuration = "ALTER TABLE FlightLeg ADD COLUMN flightDuration int"
cursor.execute(addColumn_flightDuration)
connection.commit()

In [ ]:
addColumn_flightType = "ALTER TABLE FlightLeg ADD COLUMN flightType CHAR(1)"
cursor.execute(addColumn_flightType)
connection.commit()

In [ ]:
update_data = """Update FlightLeg set flightDuration = ? where id = ?"""
for i in range(len(flightlegs.index)):
    string = flightlegs.iloc[i][0]
    data = string.split(";")
    date_start = datetime.strptime(data[5], '%Y-%m-%d %H:%M:%S')
    date_stop = datetime.strptime(data[6], '%Y-%m-%d %H:%M:%S')
    date_diff = round((date_stop - date_start).total_seconds() / 60)
    time = (date_diff,i)
    cursor.execute(update_data, time)
connection.commit()

In [ ]:
#cursor.execute('SELECT * FROM FlightLeg')
#for row in cursor.fetchall():
#     print(row)

In [ ]:
update_data = """Update FlightLeg set flightType = ? where id = ?"""
for i in range(len(flightlegs.index)):
    string = flightlegs.iloc[i][0]
    data = string.split(";")
    if data[2] == data[4]:
        code = "D"
    else:
        code = "I"
    codes = (code,i)
    cursor.execute(update_data, codes)
connection.commit()

In [ ]:
#cursor.execute('SELECT * FROM FlightLeg WHERE flightType = "D" ')
#for row in cursor.fetchall():
#     print(row)

In [ ]:
cursor.execute('SELECT tailNumber FROM FlightLeg GROUP BY tailNumber ORDER BY COUNT(tailNumber) DESC LIMIT 1')
for row in cursor.fetchall():
     print("Samolot który wykonał najwięcej lotów",row[0])

Samolot który wykonał najwięcej lotów A6-ENE


In [ ]:
cursor.execute('SELECT tailNumber FROM FlightLeg GROUP BY tailNumber ORDER BY SUM(flightDuration) DESC LIMIT 1')
for row in cursor.fetchall():
     print("Samolot który przeleciał najwięcej minut",row[0])

Samolot który przeleciał najwięcej minut A6-EVF


In [ ]:
cursor.execute('SELECT tailNumber,departureTimeUtc FROM FlightLeg WHERE flightType = "D" ORDER BY flightDuration DESC LIMIT 1')
for row in cursor.fetchall():
     print("Najdłuższy lot krajowy",row)
cursor.execute('SELECT tailNumber,departureTimeUtc FROM FlightLeg WHERE flightType = "D" ORDER BY flightDuration ASC LIMIT 1')
for row in cursor.fetchall():
     print("Najkrótszy lot krajowy",row)

Najdłuższy lot krajowy ('A6-EES', '2021-11-30 09:20:00')
Najkrótszy lot krajowy ('A6-ECH', '2021-11-30 09:32:00')


In [ ]:
cursor.execute('SELECT tailNumber,departureTimeUtc FROM FlightLeg WHERE flightType = "I" ORDER BY flightDuration DESC LIMIT 1')
for row in cursor.fetchall():
     print("Najdłuższy lot międzykrajowy",row)
cursor.execute('SELECT tailNumber,departureTimeUtc FROM FlightLeg WHERE flightType = "I" ORDER BY flightDuration ASC LIMIT 1')
for row in cursor.fetchall():
     print("Najkrótszy lot międzykrajowy",row)

Najdłuższy lot międzykrajowy ('A6-EWH', '2021-11-29 06:32:00')
Najkrótszy lot międzykrajowy ('A6-ENE', '2021-11-29 13:35:00')


In [ ]:
visited_name = []
for i in range(len(flightlegs.index)):
    counter = 0
    string = flightlegs.iloc[i][0]
    data = string.split(";")
    name = data[0]
    date_start = datetime.strptime(data[5], '%Y-%m-%d %H:%M:%S')
    date_stop = datetime.strptime(data[6], '%Y-%m-%d %H:%M:%S')
    for j in range(i+1,len(flightlegs.index)):
        string2 = flightlegs.iloc[j][0]
        data2 = string2.split(";")
        name2 = data2[0]
        if name not in visited_name:
            if name == name2:
                date_start2 = datetime.strptime(data2[5], '%Y-%m-%d %H:%M:%S')
                date_stop2 = datetime.strptime(data2[6], '%Y-%m-%d %H:%M:%S')
                if date_start >= date_start2 and date_start <= date_stop2:
                    counter += 1
                elif date_stop >= date_start2 and date_stop < date_stop2:
                    counter += 1
    visited_name.append(name)
print(counter)

0


In [ ]:
cursor.execute('SELECT tailNumber FROM FlightLeg GROUP BY tailNumber')
names = []
get_flights_by_name = """SELECT * FROM FlightLeg WHERE tailNumber = ? ORDER BY departureTimeUtc ASC"""
for row in cursor.fetchall():
     names.append(row[0])

flights = []
for name in names:
    cursor.execute(get_flights_by_name, (name,))
    for row in cursor.fetchall():
        flights.append(row)
    best = 100000
    for i in range(len(flights)):
        for j in range(i,len(flights)):
            date_start = datetime.strptime(flights[i][7], '%Y-%m-%d %H:%M:%S')
            date_stop = datetime.strptime(flights[j][6], '%Y-%m-%d %H:%M:%S')
            date_diff = round((date_stop - date_start).total_seconds() / 60)
            if date_diff < best and date_diff > 0:
                best = date_diff
                name1 = flights[i]
                name2 = flights[j]
    flights.clear()
    print(name1,name2,best)